### Métodos - Otimização CEDS 2019

Este *notebook* visa à criação de uma classe que incorpore os métodos vistos e abordados para a resolução de problemas de otimização. Sendo assim, a escolha de cada possibilidade fica a critério do usuário, o qual precisa apenas fornecer a função, o gradiente e a hessiana (quando disponíveis).

#### Importando bibliotecas úteis

In [1]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
%matplotlib inline

Testando funcionalidade de derivação:

In [2]:
x = Symbol('x')
w = Symbol('w')
y = x**2 + 2*w**2
partial_x = y.diff(x)
partial_w = y.diff(w)
print(partial_x)
print(partial_w)

2*x
4*w


#### Definindo função, gradiente e hessiana

In [3]:
def f(x):
    return x[0]**2+2*x[1]**2

def grad(x):
    output = np.zeros(x.shape[0])
    
    output[0] = 2*x[0]
    output[1] = 4*x[1]
    
    return output

def hess(x):
    output = np.zeros((x.shape[0], x.shape[0]))
    
    output[0,0] = 2
    output[0,1] = 0
    output[1,0] = 0
    output[1,1] = 4
    
    return output

#### Classe de métodos de otimização

In [4]:
class Optimization():
    '''
    Class designed for solving a minimization problem.\n
    Initialization requires:
    >>> Function
    >>> Gradient
    >>> Hessian (optional)
    >>> max_iter (optional)
    >>> precision (optional)
    '''
    
    def __init__(self, function, gradient, hessian=None, max_iter=4, precision=1e-5):
        self.f = function
        self.grad = gradient
        self.hess = hessian
        self.max_iter = max_iter
        self.precision = precision
        
    def GradientDescent(self, x0, method='cauchy', t0=None, backtracking=False, alpha=0.1):
        '''
        Gradient Descent method.
        >>> x0: initial approximation
        >>> method: ('cauchy', 'constant')
        >>> t: required if method 'constant' is selected
        >>> backtracking: (True, False)
        >>> alpha: value needed if backtracking = True
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.ts = []
        self.xs = []
        
        self.xs.append(x0)
        self.funcs.append(self.f(x0))
        self.grads.append(self.grad(x0))
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        
        if method=='cauchy':
            
            for i in range(self.max_iter):
                t = self.grads[-1]@self.grads[-1]/(self.grads[-1].T@self.hess(x0)@self.grads[-1])
                if backtracking==True:
                    while self.f(x0-t*self.grads[-1]) >= self.f(x0)-t*alpha*(self.grads[-1]@self.grads[-1]):
                        t/=2
                self.ts.append(t)
                
                x = x0 - t*self.grads[-1]
                
                self.xs.append(x)
                self.funcs.append(self.f(x))
                self.grads.append(self.grad(x))
                self.directions.append(-self.grads[-1])
                self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                x0 = x
                
                if self.grad_norms[-1] < self.precision:
                    break
                
        elif method=='constant':
            
            for i in range(self.max_iter):
                t=t0
                if backtracking==True:
                    while self.f(x0-t*self.grads[-1]) >= self.f(x0)-t*alpha*(self.grads[-1]@self.grads[-1]):
                        t=t/2
                self.ts.append(t)
                
                x = x0 - t*self.grads[-1]
                
                self.xs.append(x)
                self.funcs.append(self.f(x))
                self.grads.append(self.grad(x))
                self.directions.append(-self.grads[-1])
                self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                x0 = x
                
                if self.grad_norms[-1] < self.precision:
                    break
                
        else:
            raise ValueError('Method not found!')
                
        print('*'*50)
        print(' '*15, 'MÉTODO DO GRADIENTE')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def CGLinear(self, x0):
        '''
        Linear CG method.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.ts = []
        self.xs = []
        
        self.xs.append(x0)
        self.funcs.append(self.f(x0))
        self.grads.append(self.grad(x0))
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        
                
        for i in range(self.max_iter):                
            t = -(self.grads[-1]@self.directions[-1])/(self.directions[-1].T@self.hess(x0)@self.directions[-1])
            self.ts.append(t)
            
            x = x0 + t*self.directions[-1]
            
            self.xs.append(x)
            self.funcs.append(self.f(x))
            self.grads.append(self.grad(x))
            aux = (self.grads[-1].T@self.hess(x)@self.directions[-1])/(self.directions[-1].T@self.hess(x)@self.directions[-1])
            d = -self.grads[-1] + aux*self.directions[-1]
            self.directions.append(d)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        print('*'*50)
        print(' '*10, 'GRADIENTES CONJUGADOS (LINEAR)')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def CGNonLinear(self, x0, method='Fletcher-Reeves', t0=1, backtracking=True, alpha=0.1):
        '''
        Fletcher-Reeves CG method.
        >>> x0: initial approximation
        >>> method: Fletcher-Reeves, Polak-Ribiere, Hestenes-Stiefel
        >>> t0: initial step for backtracking
        >>> backtracking: True, False
        >>> alpha: Armijo search
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.ts = []
        self.xs = []
        
        self.xs.append(x0)
        self.funcs.append(self.f(x0))
        self.grads.append(self.grad(x0))
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        
        for i in range(self.max_iter):                
            t = t0
            if backtracking==True:
                while self.f(x0+t*self.directions[-1]) >= self.f(x0)+t*alpha*(self.grads[-1].T@self.directions[-1]):
                    t=t/2
            self.ts.append(t)
            
            x = x0 + t*self.directions[-1]
            
            self.xs.append(x)
            self.funcs.append(self.f(x))
            self.grads.append(self.grad(x))
            if method=='Fletcher-Reeves':
                beta = (self.grads[-1].T@self.grads[-1])/(self.grads[-2].T@self.grads[-2])
            elif method=='Polak-Ribiere':
                beta = (self.grads[-1].T@(self.grads[-1]-self.grads[-2]))/(self.grads[-2].T@self.grads[-2])
            elif method=='Hestenes-Stiefel':
                beta=(self.grads[-1].T@(self.grads[-1]-self.grads[-2]))/((self.grads[-1]-self.grads[-2]).T@self.directions[-1])
            else:
                raise ValueError('Method not found!')
            d = -self.grads[-1] + beta*self.directions[-1]
            self.directions.append(d)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        print('*'*50)
        print(' '*17, method.upper())
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def CD(self, x0, method='sequential', t0=1, backtracking=True, alpha=0.1):
        '''
        CD method.
        >>> x0: initial approximation
        >>> method: sequential, random
        >>> t0: initial step for backtracking
        >>> backtracking: True, False
        >>> alpha: Armijo search
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.ts = []
        self.xs = []
                
        for i in range(self.max_iter):
            
            if method=='sequential':
                j = i%len(x0)
            elif method=='random':
                j = np.random.randint(low=0, high=len(x0))
            else:
                raise ValueError('Method not found')
                
            self.xs.append(x0)
            self.funcs.append(self.f(x0))
            self.grads.append(self.grad(x0))
            aux_vec = np.zeros(self.grads[-1].shape)
            aux_vec[j] = 1
            self.directions.append(-self.grads[-1]*aux_vec)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            
            t = t0
            if backtracking==True:
                while self.f(x0+t*self.directions[-1]) >= self.f(x0)+t*alpha*(self.grads[-1].T@self.directions[-1]):
                    t=t/2
            self.ts.append(t)
            
            x = x0 + t*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        if method=='sequential':
            j = i%len(x0)
        elif method=='random':
            j = np.random.randint(low=0, high=len(x0))
        else:
            raise ValueError('Method not found')
        self.xs.append(x)
        self.funcs.append(self.f(x))
        self.grads.append(self.grad(x))
        aux_vec = np.zeros(self.grads[-1].shape)
        aux_vec[j] = 1
        self.directions.append(-self.grads[-1]*aux_vec)
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        print('*'*50)
        print(' '*17, method.upper())
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
                
    def iterations(self, n_iter):
        
        print(f'Aproximação inicial: {np.round(self.xs[0],5)}')
        print(f'Valor da função: {np.round(self.funcs[0],5)}')
        print(f'Gradiente: {np.round(self.grads[0],5)}')
        print(f'Direção: {np.round(self.directions[0],5)}')
        print(f'Norma do gradiente: {np.round(self.grad_norms[0],5)}')
        
        for i in range(n_iter):
            print('-'*50)
            print(' '*20, '{}ª iteração'.format(i+1))
            print('-'*50)
            print()
            print(f'x: {np.round(self.xs[i+1],5)}')
            print(f'Valor do passo: {np.round(self.ts[i],5)}')
            print(f'Valor da função: {np.round(self.funcs[i+1],5)}')
            print(f'Gradiente: {np.round(self.grads[i+1],5)}')
            print(f'Direção: {np.round(self.directions[i+1],5)}')
            print(f'Norma do gradiente: {np.round(self.grad_norms[i+1],5)}')
            print()

#### Testes

In [5]:
opt = Optimization(f, grad, hess)

In [6]:
x0 = np.array([1,2])

+ Passo constante: $t=0.99(2/L)$

In [7]:
opt.GradientDescent(x0, method='constant', t0=(0.99*2/4))

**************************************************
                MÉTODO DO GRADIENTE
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [ 0.01 -1.96]
Valor do passo: 0.495
Valor da função: 7.6833
Gradiente: [ 0.02 -7.84]
Direção: [-0.02  7.84]
Norma do gradiente: 7.84003

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.0000e-04 1.9208e+00]
Valor do passo: 0.495
Valor da função: 7.37895
Gradiente: [2.0000e-04 7.6832e+00]
Direção: [-2.0000e-04 -7.6832e+00]
Norma do gradiente: 7.6832

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [ 0.      -1.88238]
Valor 

+ Passo constante: $t_{0}=1$  com **BACKTRACKING**

In [8]:
opt.GradientDescent(x0, method='constant', t0=1, backtracking=True, alpha=0.1)

**************************************************
                MÉTODO DO GRADIENTE
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.5 0. ]
Valor do passo: 0.25
Valor da função: 0.25
Gradiente: [1. 0.]
Direção: [-1. -0.]
Norma do gradiente: 1.0

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0. 0.]
Valor do passo: 0.5
Valor da função: 0.0
Gradiente: [0. 0.]
Direção: [-0. -0.]
Norma do gradiente: 0.0




+ Cauchy

In [9]:
opt.GradientDescent(x0, method='cauchy')

**************************************************
                MÉTODO DO GRADIENTE
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [ 0.48485 -0.06061]
Valor do passo: 0.25758
Valor da função: 0.24242
Gradiente: [ 0.9697  -0.24242]
Direção: [-0.9697   0.24242]
Norma do gradiente: 0.99954

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.02694 0.05387]
Valor do passo: 0.47222
Valor da função: 0.00653
Gradiente: [0.05387 0.21549]
Direção: [-0.05387 -0.21549]
Norma do gradiente: 0.22212

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [ 0.01306 -0.00163]


+ Cauchy com **BACKTRACKING**

In [10]:
opt.GradientDescent(x0, method='cauchy', backtracking=True, alpha=0.1)

**************************************************
                MÉTODO DO GRADIENTE
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [ 0.48485 -0.06061]
Valor do passo: 0.25758
Valor da função: 0.24242
Gradiente: [ 0.9697  -0.24242]
Direção: [-0.9697   0.24242]
Norma do gradiente: 0.99954

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.02694 0.05387]
Valor do passo: 0.47222
Valor da função: 0.00653
Gradiente: [0.05387 0.21549]
Direção: [-0.05387 -0.21549]
Norma do gradiente: 0.22212

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [ 0.01306 -0.00163]


+ Gradientes Conjugados LINEAR

In [11]:
def fCG(x):
    return 1

def gradCG(x):
    return hessCG(x)@x

def hessCG(x):
    return np.diag([10, 20, 30, 40, 50])

In [12]:
optCG = Optimization(fCG, gradCG, hessCG, max_iter=10)
x0CG = np.array([1, 1, 1, 1, 1])
optCG.CGLinear(x0CG)

**************************************************
           GRADIENTES CONJUGADOS (LINEAR)
**************************************************

Aproximação inicial: [1 1 1 1 1]
Valor da função: 1
Gradiente: [10 20 30 40 50]
Direção: [-10 -20 -30 -40 -50]
Norma do gradiente: 74.16198
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [ 0.75556  0.51111  0.26667  0.02222 -0.22222]
Valor do passo: 0.02444
Valor da função: 1
Gradiente: [  7.55556  10.22222   8.        0.88889 -11.11111]
Direção: [ -8.1916  -11.49432  -9.90815  -3.43309   7.93086]
Norma do gradiente: 18.70367

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [ 0.46536  0.10392 -0.08434 -0.0994   0.05873]
Valor do passo: 0.03543
Valor da função: 1
Gradiente: [ 4.65361  2.07831 -2.53012 -3.9759   2.93675]
Direção: [-5.98387 -3.94491  0.92111  3.4184 

+ Fletcher-Reeves

In [13]:
opt.CGNonLinear(x0, method='Fletcher-Reeves', t0=1, backtracking=True, alpha=0.1)

**************************************************
                  FLETCHER-REEVES
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.5 0. ]
Valor do passo: 0.25
Valor da função: 0.25
Gradiente: [1. 0.]
Direção: [-1.02941 -0.11765]
Norma do gradiente: 1.0

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [-0.01471 -0.05882]
Valor do passo: 0.5
Valor da função: 0.00714
Gradiente: [-0.02941 -0.23529]
Direção: [-0.02847  0.22868]
Norma do gradiente: 0.23713

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [-0.02182 -0.00165]
Valor do passo: 0.25
Valor da funç

+ Polak-Ribiere

In [14]:
opt.CGNonLinear(x0, method='Polak-Ribiere', t0=1, backtracking=True, alpha=0.1)

**************************************************
                  POLAK-RIBIERE
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.5 0. ]
Valor do passo: 0.25
Valor da função: 0.25
Gradiente: [1. 0.]
Direção: [-0.97059  0.11765]
Norma do gradiente: 1.0

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.01471 0.05882]
Valor do passo: 0.5
Valor da função: 0.00714
Gradiente: [0.02941 0.23529]
Direção: [-0.05544 -0.23214]
Norma do gradiente: 0.23713

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [0.00085 0.00079]
Valor do passo: 0.25
Valor da função: 0.0


+ Hestenes-Stiefel

In [15]:
opt.CGNonLinear(x0, method='Hestenes-Stiefel', t0=1, backtracking=True, alpha=0.1)

**************************************************
                  HESTENES-STIEFEL
**************************************************

Aproximação inicial: [1 2]
Valor da função: 9
Gradiente: [2. 8.]
Direção: [-2. -8.]
Norma do gradiente: 8.24621
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.5 0. ]
Valor do passo: 0.25
Valor da função: 0.25
Gradiente: [1. 0.]
Direção: [-0.9697   0.12121]
Norma do gradiente: 1.0

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.01515 0.06061]
Valor do passo: 0.5
Valor da função: 0.00758
Gradiente: [0.0303  0.24242]
Direção: [-0.05969 -0.23875]
Norma do gradiente: 0.24431

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [0.00023 0.00092]
Valor do passo: 0.25
Valor da função: 0

**Descenso Coordenado**

In [16]:
def fDC(x):
    x1, x2, x3 = x
    return np.exp(x1-x3)*((x2-1)**2)+x1**2+x2**2+x3**2

def gradDC(x):
    output = np.zeros(3)
    x1, x2, x3 = x
    output[0] = np.exp(x1-x3)*((x2-1)**2)+2*x1
    output[1] = np.exp(x1-x3)*2*(x2-1)+2*x2
    output[2] = -np.exp(x1-x3)*((x2-1)**2)+2*x3
    
    return output

+ Sequencial

In [17]:
optDC = Optimization(fDC, gradDC, max_iter=4)
x0DC = np.array([2, 1, 0])
optDC.CD(x0DC, method='sequential', t0=0.1, backtracking=False)

**************************************************
                  SEQUENTIAL
**************************************************

Aproximação inicial: [2 1 0]
Valor da função: 5.0
Gradiente: [4. 2. 0.]
Direção: [-4. -0. -0.]
Norma do gradiente: 4.47214
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.6 1.  0. ]
Valor do passo: 0.1
Valor da função: 3.56
Gradiente: [3.2 2.  0. ]
Direção: [-0. -2. -0.]
Norma do gradiente: 3.77359

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.6 0.8 0. ]
Valor do passo: 0.1
Valor da função: 3.39812
Gradiente: [ 3.39812 -0.38121 -0.19812]
Direção: [-0.       0.       0.19812]
Norma do gradiente: 3.42517

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [1.6     0.8     0.01981]
Val

+ Randomizado

In [18]:
optDC = Optimization(fDC, gradDC, max_iter=4)
x0DC = np.array([2, 1, 0])
optDC.CD(x0DC, method='random', t0=0.1, backtracking=False)

**************************************************
                  RANDOM
**************************************************

Aproximação inicial: [2 1 0]
Valor da função: 5.0
Gradiente: [4. 2. 0.]
Direção: [-0. -0. -0.]
Norma do gradiente: 4.47214
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [2. 1. 0.]
Valor do passo: 0.1
Valor da função: 5.0
Gradiente: [4. 2. 0.]
Direção: [-0. -0. -0.]
Norma do gradiente: 4.47214

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [2. 1. 0.]
Valor do passo: 0.1
Valor da função: 5.0
Gradiente: [4. 2. 0.]
Direção: [-0. -2. -0.]
Norma do gradiente: 4.47214

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [2.  0.8 0. ]
Valor do passo: 0.1
Valor da função: 4.93556
Gradiente: [ 4.29556 

### Funções com dados

In [19]:
def f(x, points):
    output = []
    for point in points:
        a, b = point
        output.append((x[0]-a)**2 + (x[1]-b)**2)
        
    return np.mean(np.array(output))

def grad(x, points):
    output = np.zeros(len(x))
    
    for point in points:
        a, b = point   
        output[0] += 2*(x[0]-a)
        output[1] += 2*(x[1]-b)
    
    return output/len(points)

def hess(x, points):
    output = np.zeros((len(x), len(x)))
    
    output[0,0] = 2
    output[0,1] = 0
    output[1,0] = 0
    output[1,1] = 2
    
    return output

#### Classe de métodos de otimização

In [20]:
class Optimization():
    '''
    Class designed for solving a minimization problem.\n
    Initialization requires:
    >>> Function
    >>> Gradient
    >>> Hessian (optional)
    >>> max_iter (optional)
    >>> precision (optional)
    '''
    
    def __init__(self, function, gradient, hessian=None, max_iter=10, precision=1e-5):
        self.f = function
        self.grad = gradient
        self.hess = hessian
        self.max_iter = max_iter
        self.precision = precision
        
    def StochasticGradient(self, x0, points, t=0.1):
        '''
        Stochastic Gradient Descent method.
        >>> x0: initial approximation
        >>> t: method step
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))         
            k = np.random.randint(low=0, high=len(points))
            self.grads.append(self.grad(x0, [points[k]]))
            self.directions.append(-self.grads[-1])
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.samples.append(points[k])
            
            x = x0 + t*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x, points))         
        k = np.random.randint(low=0, high=len(points))
        self.grads.append(self.grad(x, [points[k]]))
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.samples.append(points[k])
                
        print('*'*50)
        print(' '*15, 'GRADIENTE ESTOCÁSTICO')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def miniBatch(self, x0, points, batch_size=2, t=0.1):
        '''
        Mini-Batch Gradient Descent method.
        >>> x0: initial approximation
        >>> t: method step
        >>> batch_size: number of points considered in each batch
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            minibatch = [points[k] for k in np.random.randint(low=0, high=len(points), size=batch_size)]
            self.grads.append(self.grad(x0, minibatch))
            self.directions.append(-self.grads[-1])
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.samples.append(minibatch)
            
            x = x0 + t*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x, points))         
        minibatch = [points[k] for k in np.random.randint(low=0, high=len(points), size=batch_size)]
        self.grads.append(self.grad(x0, minibatch))
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.samples.append(minibatch)
                
        print('*'*50)
        print(' '*15, 'GRADIENTE MINI-BATCH')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def SVRG(self, x0, points, base_case=5, t=0.1):
        '''
        Stochastic Gradient with Variance Reduction.
        >>> x0: initial approximation
        >>> t: method step
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
                   
        for i in range(self.max_iter):
            if i==0 or i%base_case==0:
                ponto_base = x0
                grad_base = self.grad(x0, points)
                self.grads.append(grad_base)
                self.samples.append(points)
            else:
                k = np.random.randint(low=0, high=len(points))
                grad_k = self.grad(x0, [points[k]])
                grad_k_base = self.grad(ponto_base, [points[k]])                               
                self.grads.append(grad_k-grad_k_base+grad_base)
                self.samples.append(points[k])
                
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            self.directions.append(-self.grads[-1])
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + t*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        if i==0 or i%base_case==0:
            ponto_base = x
            grad_base = self.grad(x, points)
            self.grads.append(grad_base)
            self.samples.append(points)
        else:
            k = np.random.randint(low=0, high=len(points))
            grad_k = self.grad(x, [points[k]])
            grad_k_base = self.grad(ponto_base, [points[k]])                               
            self.grads.append(grad_k-grad_k_base+grad_base)
            self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))         
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MÉTODO SVRG')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def SAG(self, x0, points, t=0.1):
        '''
        SAG.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        grad_k = [np.zeros(len(x0)) for i in range(len(points))]
                   
        for i in range(self.max_iter):
            if i==0:
                for k in range(len(points)):
                    grad_k[k] = self.grad(x0, [points[k]])
                self.grads.append(self.grad(x0, points))
                self.samples.append(points)
            else:
                k = np.random.randint(low=0, high=len(points))
                grad_k[k] = self.grad(x0, [points[k]])
                grad_sag = np.mean(np.array(grad_k), axis=0)
                self.grads.append(grad_sag)
                self.samples.append(points[k])
                
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            self.directions.append(-self.grads[-1])
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + t*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        grad_k[k] = self.grad(x, [points[k]])
        grad_sag = np.mean(np.array(grad_k), axis=0)
        self.grads.append(grad_sag)
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))         
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MÉTODO SAG')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def SAGA(self, x0, points, t=0.1):
        '''
        SAGA.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        grad_k = [np.zeros(len(x0)) for i in range(len(points))]
                   
        for i in range(self.max_iter):
            if i==0:
                for k in range(len(points)):
                    grad_k[k] = self.grad(x0, [points[k]])
                self.grads.append(self.grad(x0, points))
                self.samples.append(points)
            else:
                k = np.random.randint(low=0, high=len(points))
                grad_k_new = self.grad(x0, [points[k]])
                grad_saga = np.mean(np.array(grad_k), axis=0)
                self.grads.append(grad_saga+grad_k_new-grad_k[k])
                grad_k[k] = grad_k_new
                self.samples.append(points[k])
                
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            self.directions.append(-self.grads[-1])
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + t*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        grad_k_new = self.grad(x0, [points[k]])
        grad_saga = np.mean(np.array(grad_k), axis=0)
        self.grads.append(grad_saga+grad_k_new-grad_k[k])
        grad_k[k] = grad_k_new
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))         
        self.directions.append(-self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MÉTODO SAGA')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def Adagrad(self, x0, points, eta=0.01, epsilon=1e-8):
        '''
        Adagrad.
        >>> x0: initial approximation
        >>> eta: adaptative step parameter
        >>> epsilon: parameter to avoid division by zero
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        G = np.zeros(len(x0))
                   
        for i in range(self.max_iter):
            
            k = np.random.randint(low=0, high=len(points))
            self.grads.append(self.grad(x0, [points[k]]))
            self.samples.append(points[k])
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            G += self.grads[-1]**2
            d = (eta/np.sqrt(G+epsilon))*self.grads[-1]
            self.directions.append(-d)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        self.grads.append(self.grad(x, [points[k]]))
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))
        G += self.grads[-1]**2
        d = (eta/np.sqrt(G+epsilon))*self.grads[-1]
        self.directions.append(-d)
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MÉTODO ADAGRAD')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def RMSProp(self, x0, points, eta=0.01, gamma=0.9, epsilon=1e-8):
        '''
        RMSProp.
        >>> x0: initial approximation
        >>> eta: adaptative step parameter
        >>> epsilon: parameter to avoid division by zero
        >>> gamma: weight assigned to past information provided by gradients
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        G = np.zeros(len(x0))
                   
        for i in range(self.max_iter):
            
            k = np.random.randint(low=0, high=len(points))
            self.grads.append(self.grad(x0, [points[k]]))
            self.samples.append(points[k])
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            G = gamma*G + (1-gamma)*self.grads[-1]**2
            d = (eta/np.sqrt(G+epsilon))*self.grads[-1]
            self.directions.append(-d)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        self.grads.append(self.grad(x, [points[k]]))
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))
        G = gamma*G + (1-gamma)*self.grads[-1]**2
        d = (eta/np.sqrt(G+epsilon))*self.grads[-1]
        self.directions.append(-d)
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MÉTODO RMSPROP')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def Polyak(self, x0, points, gamma=0.1, t=0.1):
        '''
        Moment - Polyak.
        >>> x0: initial approximation
        >>> t: method step
        >>> gamma: weight assigned to past information provided by gradients
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        m = np.zeros(len(x0))
                   
        for i in range(self.max_iter):
            
            k = np.random.randint(low=0, high=len(points))
            self.grads.append(self.grad(x0, [points[k]]))
            self.samples.append(points[k])
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            m = gamma*m + t*self.grads[-1]
            self.directions.append(-m)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        self.grads.append(self.grad(x, [points[k]]))
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))
        m = gamma*m + t*self.grads[-1]
        self.directions.append(-m)
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MOMENTO DE POLYAK')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def Nesterov(self, x0, points, gamma=0.1, t=0.1):
        '''
        Moment - Nesterov.
        >>> x0: initial approximation
        >>> t: method step
        >>> gamma: weight assigned to past information provided by gradients
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        m = np.zeros(len(x0))
                   
        for i in range(self.max_iter):
            
            k = np.random.randint(low=0, high=len(points))
            self.samples.append(points[k])
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            z = x0+gamma*m
            self.grads.append(self.grad(z, [points[k]]))
            self.directions.append(-t*self.grads[-1])
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = z + self.directions[-1]
            m = x-x0
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))
        z = x+gamma*m
        self.grads.append(self.grad(z, [points[k]]))
        self.directions.append(-t*self.grads[-1])
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MOMENTO DE NESTEROV')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def ADAM(self, x0, points, eta=0.01, gamma1=0.9, gamma2=0.999, epsilon=1e-8):
        '''
        ADAM.
        >>> x0: initial approximation
        >>> eta: adaptative step parameter
        >>> epsilon: parameter to avoid division by zero
        >>> gamma1: moving average parameter
        >>> gamma2: weight assigned to past information provided by gradients
        '''
        
        self.funcs = []
        self.grads = []
        self.directions = []
        self.grad_norms = []
        self.samples = []
        self.xs = []
        
        G = np.zeros(len(x0))
        m = np.zeros(len(x0))
                   
        for i in range(self.max_iter):
            
            k = np.random.randint(low=0, high=len(points))
            self.grads.append(self.grad(x0, [points[k]]))
            self.samples.append(points[k])
            self.xs.append(x0)
            self.funcs.append(self.f(x0, points))
            G = gamma2*G + (1-gamma2)*self.grads[-1]**2
            G_hat = G/(1-gamma2)
            m = gamma1*m + (1-gamma1)*self.grads[-1]
            m_hat = m/(1-gamma1)
            self.directions.append(-m_hat)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
                       
            x = x0 + (eta/(np.sqrt(G_hat)+epsilon))*self.directions[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
        
        k = np.random.randint(low=0, high=len(points))
        self.grads.append(self.grad(x, [points[k]]))
        self.samples.append(points[k])
        self.xs.append(x)
        self.funcs.append(self.f(x, points))
        G = gamma2*G + (1-gamma2)*self.grads[-1]**2
        G_hat = G/(1-gamma2)
        m = gamma1*m + (1-gamma1)*self.grads[-1]
        m_hat = m/(1-gamma1)
        self.directions.append(-m_hat)
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))

        print('*'*50)
        print(' '*15, 'MÉTODO ADAM')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
                       
    def iterations(self, n_iter):
        
        print(f'Aproximação inicial: {np.round(self.xs[0],5)}')
        print('Ponto selecionado:\n', np.round(self.samples[0],5))
        print(f'Valor da função: {np.round(self.funcs[0],5)}')
        print(f'Gradiente: {np.round(self.grads[0],5)}')
        print(f'Direção: {np.round(self.directions[0],5)}')
        print(f'Norma do gradiente: {np.round(self.grad_norms[0],5)}')
        
        for i in range(n_iter):
            print('-'*50)
            print(' '*20, '{}ª iteração'.format(i+1))
            print('-'*50)
            print()
            print(f'x: {np.round(self.xs[i+1],5)}')
            print('Ponto selecionado:\n', np.round(self.samples[i+1],5))
            print(f'Valor da função: {np.round(self.funcs[i+1],5)}')
            print(f'Gradiente: {np.round(self.grads[i+1],5)}')
            print(f'Direção: {np.round(self.directions[i+1],5)}')
            print(f'Norma do gradiente: {np.round(self.grad_norms[i+1],5)}')
            print()

#### Testes

In [21]:
opt = Optimization(f, grad, hess)

In [22]:
x0 = np.array([2,3])
points = [(1,0), (0,1), (-1,0), (0,-1)]

+ Gradiente Estocástico

In [23]:
opt.StochasticGradient(x0, points, t=0.1)

**************************************************
                GRADIENTE ESTOCÁSTICO
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [0 1]
Valor da função: 14.0
Gradiente: [4. 4.]
Direção: [-4. -4.]
Norma do gradiente: 5.65685
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.6 2.6]
Ponto selecionado:
 [ 0 -1]
Valor da função: 10.32
Gradiente: [3.2 7.2]
Direção: [-3.2 -7.2]
Norma do gradiente: 7.87909

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.28 1.88]
Ponto selecionado:
 [1 0]
Valor da função: 6.1728
Gradiente: [0.56 3.76]
Direção: [-0.56 -3.76]
Norma do gradiente: 3.80147

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [1.224 1.504]
Ponto selecionado:

+ Gradiente Estocástico - MiniBatch

In [24]:
opt.miniBatch(x0, points, batch_size=2, t=0.1)

**************************************************
                GRADIENTE MINI-BATCH
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [[ 0 -1]
 [ 1  0]]
Valor da função: 14.0
Gradiente: [3. 7.]
Direção: [-3. -7.]
Norma do gradiente: 7.61577
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.7 2.3]
Ponto selecionado:
 [[ 1  0]
 [-1  0]]
Valor da função: 9.18
Gradiente: [3.4 4.6]
Direção: [-3.4 -4.6]
Norma do gradiente: 5.72014

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.36 1.84]
Ponto selecionado:
 [[-1  0]
 [-1  0]]
Valor da função: 6.2352
Gradiente: [4.72 3.68]
Direção: [-4.72 -3.68]
Norma do gradiente: 5.98505

--------------------------------------------------
                     3ª iteração
--------------------------------------------------



+ Gradiente Estocástico - SVRG

In [25]:
opt.SVRG(x0, points, base_case=5, t=0.1)

**************************************************
                MÉTODO SVRG
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [[ 1  0]
 [ 0  1]
 [-1  0]
 [ 0 -1]]
Valor da função: 14.0
Gradiente: [4. 6.]
Direção: [-4. -6.]
Norma do gradiente: 7.2111
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.6 2.4]
Ponto selecionado:
 [ 0 -1]
Valor da função: 9.32
Gradiente: [3.2 4.8]
Direção: [-3.2 -4.8]
Norma do gradiente: 5.76888

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.28 1.92]
Ponto selecionado:
 [ 0 -1]
Valor da função: 6.3248
Gradiente: [2.56 3.84]
Direção: [-2.56 -3.84]
Norma do gradiente: 4.61511

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [1.024 1.53

+ Gradiente Estocástico - SAG

In [26]:
opt.SAG(x0, points, t=0.1)

**************************************************
                MÉTODO SAG
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [[ 1  0]
 [ 0  1]
 [-1  0]
 [ 0 -1]]
Valor da função: 14.0
Gradiente: [4. 6.]
Direção: [-4. -6.]
Norma do gradiente: 7.2111
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.6 2.4]
Ponto selecionado:
 [ 0 -1]
Valor da função: 9.32
Gradiente: [3.8 5.7]
Direção: [-3.8 -5.7]
Norma do gradiente: 6.85055

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.22 1.83]
Ponto selecionado:
 [0 1]
Valor da função: 5.8373
Gradiente: [3.41  5.115]
Direção: [-3.41  -5.115]
Norma do gradiente: 6.14746

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [0.879  1.

+ Gradiente Estocástico - SAGA

In [27]:
opt.SAGA(x0, points, t=0.1)

**************************************************
                MÉTODO SAGA
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [[ 1  0]
 [ 0  1]
 [-1  0]
 [ 0 -1]]
Valor da função: 14.0
Gradiente: [4. 6.]
Direção: [-4. -6.]
Norma do gradiente: 7.2111
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.6 2.4]
Ponto selecionado:
 [0 1]
Valor da função: 9.32
Gradiente: [3.2 4.8]
Direção: [-3.2 -4.8]
Norma do gradiente: 5.76888

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.28 1.92]
Ponto selecionado:
 [0 1]
Valor da função: 6.3248
Gradiente: [3.16 4.74]
Direção: [-3.16 -4.74]
Norma do gradiente: 5.69677

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [0.964 1.446]
P

+ Adagrad

In [28]:
opt.Adagrad(x0, points, eta=0.01, epsilon=1e-8)

**************************************************
                MÉTODO ADAGRAD
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [-1  0]
Valor da função: 14.0
Gradiente: [6. 6.]
Direção: [-0.01 -0.01]
Norma do gradiente: 8.48528
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.99 2.99]
Ponto selecionado:
 [ 0 -1]
Valor da função: 13.9002
Gradiente: [3.98 7.98]
Direção: [-0.00553 -0.00799]
Norma do gradiente: 8.91744

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.98447 2.98201]
Ponto selecionado:
 [1 0]
Valor da função: 13.8305
Gradiente: [1.96894 5.96401]
Direção: [-0.00264 -0.00513]
Norma do gradiente: 6.28062

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: 

+ RMSProp

In [29]:
opt.RMSProp(x0, points, eta=0.01, gamma=0.9, epsilon=1e-8)

**************************************************
                MÉTODO RMSPROP
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [ 0 -1]
Valor da função: 14.0
Gradiente: [4. 8.]
Direção: [-0.03162 -0.03162]
Norma do gradiente: 8.94427
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.96838 2.96838]
Ponto selecionado:
 [1 0]
Valor da função: 13.68577
Gradiente: [1.93675 5.93675]
Direção: [-0.01438 -0.01948]
Norma do gradiente: 6.24468

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.954   2.94889]
Ponto selecionado:
 [ 0 -1]
Valor da função: 13.5141
Gradiente: [3.908   7.89779]
Direção: [-0.02198 -0.02067]
Norma do gradiente: 8.81178

--------------------------------------------------
                     3ª iteração
------------------------------------

+ Momento de Polyak

In [30]:
opt.Polyak(x0, points, gamma=0.1, t=0.1)

**************************************************
                MOMENTO DE POLYAK
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [-1  0]
Valor da função: 14.0
Gradiente: [6. 6.]
Direção: [-0.6 -0.6]
Norma do gradiente: 8.48528
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.4 2.4]
Ponto selecionado:
 [1 0]
Valor da função: 8.72
Gradiente: [0.8 4.8]
Direção: [-0.14 -0.54]
Norma do gradiente: 4.86621

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.26 1.86]
Ponto selecionado:
 [ 0 -1]
Valor da função: 6.0472
Gradiente: [2.52 5.72]
Direção: [-0.266 -0.626]
Norma do gradiente: 6.2505

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [0.994 1.234]
Ponto selecionad

+ Momento de Nesterov

In [31]:
opt.Nesterov(x0, points, gamma=0.1, t=0.1)

**************************************************
                MOMENTO DE NESTEROV
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [1 0]
Valor da função: 14.0
Gradiente: [2. 6.]
Direção: [-0.2 -0.6]
Norma do gradiente: 6.32456
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.8 2.4]
Ponto selecionado:
 [-1  0]
Valor da função: 10.0
Gradiente: [5.56 4.68]
Direção: [-0.556 -0.468]
Norma do gradiente: 7.26746

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.224 1.872]
Ponto selecionado:
 [-1  0]
Valor da função: 6.00256
Gradiente: [4.3328 3.6384]
Direção: [-0.43328 -0.36384]
Norma do gradiente: 5.65784

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [0.73312 1.4

+ Momento ADAM

In [32]:
opt.ADAM(x0, points, eta=0.01, gamma1=0.9, gamma2=0.999, epsilon=1e-8)

**************************************************
                MÉTODO ADAM
**************************************************

Aproximação inicial: [2 3]
Ponto selecionado:
 [1 0]
Valor da função: 14.0
Gradiente: [2. 6.]
Direção: [-2. -6.]
Norma do gradiente: 6.32456
--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [1.99 2.99]
Ponto selecionado:
 [1 0]
Valor da função: 13.9002
Gradiente: [1.98 5.98]
Direção: [ -3.78 -11.38]
Norma do gradiente: 6.29927

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [1.97657 2.97656]
Ponto selecionado:
 [ 0 -1]
Valor da função: 13.76674
Gradiente: [3.95313 7.95313]
Direção: [ -7.35513 -18.19513]
Norma do gradiente: 8.88141

--------------------------------------------------
                     3ª iteração
--------------------------------------------------

x: [1.9614 2.

### Métodos de Segunda Ordem

In [33]:
def f(x):
    x1, x2 = x
    return np.exp(x1+x2)+x1**2+x2**2

def grad(x):
    x1, x2 = x
    output = np.zeros(len(x))
    output[0] = np.exp(x1+x2)+2*x1
    output[1] = np.exp(x1+x2)+2*x2
    
    return output

def hess(x):
    x1, x2 = x
    output = np.zeros((len(x), len(x)))
    output[0,0] = np.exp(x1+x2)+2
    output[0,1] = np.exp(x1+x2)
    output[1,0] = np.exp(x1+x2)
    output[1,1] = np.exp(x1+x2)+2
    
    return output

#### Classe de métodos de otimização

In [34]:
class Optimization():
    '''
    Class designed for solving a minimization problem.\n
    Initialization requires:
    >>> Function
    >>> Gradient
    >>> Hessian (optional)
    >>> max_iter (optional)
    >>> precision (optional)
    '''
    
    def __init__(self, function, gradient, hessian=None, max_iter=10, precision=1e-5):
        self.f = function
        self.grad = gradient
        self.hess = hessian
        self.max_iter = max_iter
        self.precision = precision
        
    def Newton(self, x0):
        '''
        Newton for minimization problems.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.Ms = []
        self.grad_norms = []
        self.ds = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0))         
            self.grads.append(self.grad(x0))
            self.Ms.append(self.hess(x0))
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
            
            x = x0 + self.ds[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x))         
        self.grads.append(self.grad(x))
        self.Ms.append(self.hess(x))
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
                
        print('*'*50)
        print(' '*15, 'MÉTODO DE NEWTON')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def BFGS(self, x0):
        '''
        BFGS for minimization problems.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.Ms = []
        self.grad_norms = []
        self.ds = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0))         
            self.grads.append(self.grad(x0))
            if i==0:
                self.Ms.append(self.hess(x0))
            else:
                s = (self.xs[-1] - self.xs[-2]).reshape(-1,1)
                y = (self.grads[-1] - self.grads[-2]).reshape(-1,1)
                M_new = self.Ms[-1] + (y@y.T)/(y.T@s) - (self.Ms[-1]@s@s.T@self.Ms[-1])/(s.T@self.Ms[-1]@s)
                self.Ms.append(M_new)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
            
            x = x0 + self.ds[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x))         
        self.grads.append(self.grad(x))
        self.Ms.append(self.hess(x))
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
                
        print('*'*50)
        print(' '*15, 'MÉTODO BFGS')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
        
    def DFP(self, x0):
        '''
        DFP for minimization problems.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.Ms = []
        self.grad_norms = []
        self.ds = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0))         
            self.grads.append(self.grad(x0))
            if i==0:
                hess_inv = np.linalg.inv(self.hess(x0))
                self.Ms.append(hess_inv)
            else:
                s = (self.xs[-1] - self.xs[-2]).reshape(-1,1)
                y = (self.grads[-1] - self.grads[-2]).reshape(-1,1)
                M_new = self.Ms[-1] + (s@s.T)/(y.T@s) - (self.Ms[-1]@y@y.T@self.Ms[-1])/(y.T@self.Ms[-1]@y)
                self.Ms.append(M_new)
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.ds.append(self.Ms[-1]@(-self.grads[-1]))
            
            x = x0 + self.ds[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x))         
        self.grads.append(self.grad(x))
        s = (self.xs[-1] - self.xs[-2]).reshape(-1,1)
        y = (self.grads[-1] - self.grads[-2]).reshape(-1,1)
        M_new = self.Ms[-1] + (s@s.T)/(y.T@s) - (self.Ms[-1]@y@y.T@self.Ms[-1])/(y.T@self.Ms[-1]@y)
        self.Ms.append(M_new)
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.ds.append(self.Ms[-1]@(-self.grads[-1]))
                
        print('*'*50)
        print(' '*15, 'MÉTODO DFP')
        print('*'*50)
        print()
        self.iterations(i+1, display=2)
        print()
        
    def Broyden(self, x0, method=1):
        '''
        DFP for minimization problems.
        >>> x0: initial approximation
        >>> method: (1, 2)
        '''
        
        self.funcs = []
        self.grads = []
        self.Ms = []
        self.grad_norms = []
        self.ds = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0))         
            self.grads.append(self.grad(x0))
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            if i==0:
                if method==1:
                    self.Ms.append(self.hess(x0))
                else:
                    hess_inv = np.linalg.inv(self.hess(x0))
                    self.Ms.append(hess_inv)
                self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
            else:
                s = (self.xs[-1] - self.xs[-2]).reshape(-1,1)
                y = (self.grads[-1] - self.grads[-2]).reshape(-1,1)
                if method==1:
                    M_new = self.Ms[-1] + ((y-self.Ms[-1]@s)@s.T)/(s.T@s)
                else:
                    M_new = self.Ms[-1] + ((s-self.Ms[-1]@y)@y.T)/(y.T@y)
                self.Ms.append(M_new)
                self.ds.append(self.Ms[-1]@(-self.grads[-1]))
            
            x = x0 + self.ds[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x))         
        self.grads.append(self.grad(x))
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        s = (self.xs[-1] - self.xs[-2]).reshape(-1,1)
        y = (self.grads[-1] - self.grads[-2]).reshape(-1,1)
        if method==1:
            M_new = self.Ms[-1] + ((y-self.Ms[-1]@s)@s.T)/(s.T@s)
        else:
            M_new = self.Ms[-1] + ((s-self.Ms[-1]@y)@y.T)/(y.T@y)
        self.Ms.append(M_new)
        self.ds.append(self.Ms[-1]@(-self.grads[-1]))

        print('*'*50)
        print(' '*15, 'MÉTODO DE BROYDEN {}'.format(method))
        print('*'*50)
        print()
        if method==1:
            self.iterations(i+1, display=1)
        else:
            self.iterations(i+1, display=2)
        print()
        
    def BarzilaiBorwein(self, x0):
        '''
        Barzilai-Borwein for minimization problems.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.Ms = []
        self.grad_norms = []
        self.ds = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0))         
            self.grads.append(self.grad(x0))
            if i==0:
                t = (self.grads[-1].T@self.grads[-1])/(self.grads[-1].T@self.hess(x0)@self.grads[-1])
                while self.f(x0-t*self.grads[-1]) >= self.f(x0)-0.1*t*(self.grads[-1].T@self.grads[-1]):
                    t/=2
            else:
                s = (self.xs[-1] - self.xs[-2])
                y = (self.grads[-1] - self.grads[-2])
                t = (s.T@s)/(s.T@y)
            self.Ms.append(1/t*np.eye(len(x0)))
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.ds.append(-t*self.grads[-1])
            
            x = x0 + self.ds[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x))         
        self.grads.append(self.grad(x))
        s = (self.xs[-1] - self.xs[-2]).reshape(-1,1)
        y = (self.grads[-1] - self.grads[-2]).reshape(-1,1)
        t = (s.T@s)/(s.T@y)
        self.Ms.append(1/t*np.eye(len(x0)))
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.ds.append(-t*self.grads[-1])
        
        print('*'*50)
        print(' '*15, 'BARZILAI-BORWEIN')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()
    def LevenbergMarquardt(self, x0, rho=0):
        '''
        Levenberg-Marquardt for minimization problems.
        >>> x0: initial approximation
        '''
        
        self.funcs = []
        self.grads = []
        self.Ms = []
        self.grad_norms = []
        self.ds = []
        self.xs = []
                   
        for i in range(self.max_iter):
            self.xs.append(x0)
            self.funcs.append(self.f(x0))         
            self.grads.append(self.grad(x0))
            hess_approx = self.grads[-1].reshape(-1,1).T@self.grads[-1].reshape(-1,1)
            self.Ms.append(hess_approx+rho*np.eye(len(x0)))
            self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
            self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
            
            x = x0 + self.ds[-1]
            x0 = x
            
            if self.grad_norms[-1] < self.precision:
                break
                
        self.xs.append(x)
        self.funcs.append(self.f(x))         
        self.grads.append(self.grad(x))
        hess_approx = self.grads[-1].reshape(-1,1).T@self.grads[-1].reshape(-1,1)
        self.Ms.append(hess_approx+rho*np.eye(len(x0)))
        self.grad_norms.append(np.linalg.norm(self.grads[-1], ord=2))
        self.ds.append(np.linalg.solve(self.Ms[-1], -self.grads[-1]))
                
        print('*'*50)
        print(' '*15, 'LEVENBERG-MARQUARDT')
        print('*'*50)
        print()
        self.iterations(i+1)
        print()

    def iterations(self, n_iter, display=1):
        
        print(f'Aproximação inicial: {np.round(self.xs[0],5)}')
        print(f'Valor da função: {np.round(self.funcs[0],5)}')
        print(f'Gradiente: {np.round(self.grads[0],5)}')
        print(f'Norma do gradiente: {np.round(self.grad_norms[0],5)}')
        print()
        print('Sistema:')
        if display==1:
            print(f'{np.round(self.Ms[0],5)} dk = {-np.round(self.grads[0],5)}')
        else:
            print(f'dk = {np.round(self.Ms[0],5)} {-np.round(self.grads[0],5)}')
        print(f'dk: {np.round(self.ds[0],5)}')
        print()
        
        for i in range(n_iter):
            print('-'*50)
            print(' '*20, '{}ª iteração'.format(i+1))
            print('-'*50)
            print()
            print(f'x: {np.round(self.xs[i+1],5)}')
            print(f'Valor da função: {np.round(self.funcs[i+1],5)}')
            print(f'Gradiente: {np.round(self.grads[i+1],5)}')
            print(f'Norma do gradiente: {np.round(self.grad_norms[i+1],5)}')
            print()
            print('Sistema:')
            if display==1:
                print(f'{np.round(self.Ms[i+1],5)} dk = {-np.round(self.grads[i+1],5)}')
            else:
                print(f'dk = {np.round(self.Ms[0],5)} {-np.round(self.grads[0],5)}')
            print(f'dk: {np.round(self.ds[i+1],5)}')
            print()

#### Testes

In [35]:
opt = Optimization(f, grad, hess)

In [36]:
x0 = np.array([1,2])

+ Método de Newton

In [37]:
opt.Newton(x0)

**************************************************
                MÉTODO DE NEWTON
**************************************************

Aproximação inicial: [1 2]
Valor da função: 25.08554
Gradiente: [22.08554 24.08554]
Norma do gradiente: 32.67849

Sistema:
[[22.08554 20.08554]
 [20.08554 22.08554]] dk = [-22.08554 -24.08554]
dk: [-0.04743 -1.04743]

--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.95257 0.95257]
Valor da função: 8.5352
Gradiente: [8.62555 8.62555]
Norma do gradiente: 12.19837

Sistema:
[[8.7204 6.7204]
 [6.7204 8.7204]] dk = [-8.62555 -8.62555]
dk: [-0.55862 -0.55862]

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.39395 0.39395]
Valor da função: 2.50919
Gradiente: [2.9867 2.9867]
Norma do gradiente: 4.22383

Sistema:
[[4.19879 2.19879]
 [2.19879 4.19879]] dk = [-2.9867 -2.9867]
dk:

+ BFGS

In [38]:
opt.BFGS(x0)

**************************************************
                MÉTODO BFGS
**************************************************

Aproximação inicial: [1 2]
Valor da função: 25.08554
Gradiente: [22.08554 24.08554]
Norma do gradiente: 32.67849

Sistema:
[[22.08554 20.08554]
 [20.08554 22.08554]] dk = [-22.08554 -24.08554]
dk: [-0.04743 -1.04743]

--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.95257 0.95257]
Valor da função: 8.5352
Gradiente: [8.62555 8.62555]
Norma do gradiente: 12.19837

Sistema:
[[14.28543 12.20371]
 [12.20371 14.20741]] dk = [-8.62555 -8.62555]
dk: [-0.31989 -0.33234]

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.63269 0.62023]
Valor da função: 4.28552
Gradiente: [4.76591 4.741  ]
Norma do gradiente: 6.72243

Sistema:
[[6.95893 4.91529]
 [4.91529 6.9573 ]] dk = [-4.76591 -4.741

+ DFP

In [39]:
opt.DFP(x0)

**************************************************
                MÉTODO DFP
**************************************************

Aproximação inicial: [1 2]
Valor da função: 25.08554
Gradiente: [22.08554 24.08554]
Norma do gradiente: 32.67849

Sistema:
dk = [[ 0.26186 -0.23814]
 [-0.23814  0.26186]] [-22.08554 -24.08554]
dk: [-0.04743 -1.04743]

--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.95257 0.95257]
Valor da função: 8.5352
Gradiente: [8.62555 8.62555]
Norma do gradiente: 12.19837

Sistema:
dk = [[ 0.26186 -0.23814]
 [-0.23814  0.26186]] [-22.08554 -24.08554]
dk: [-0.31628 -0.33548]

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.63629 0.61709]
Valor da função: 4.28785
Gradiente: [4.77476 4.73636]
Norma do gradiente: 6.72543

Sistema:
dk = [[ 0.26186 -0.23814]
 [-0.23814  0.26186]] [-22.08554 

+ Barzilai-Borwein

In [40]:
opt.BarzilaiBorwein(x0)

**************************************************
                BARZILAI-BORWEIN
**************************************************

Aproximação inicial: [1 2]
Valor da função: 25.08554
Gradiente: [22.08554 24.08554]
Norma do gradiente: 32.67849

Sistema:
[[42.09584  0.     ]
 [ 0.      42.09584]] dk = [-22.08554 -24.08554]
dk: [-0.52465 -0.57216]

--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.47535 1.42784]
Valor da função: 8.97195
Gradiente: [7.65797 9.56295]
Norma do gradiente: 12.2513

Sistema:
[[26.34922  0.     ]
 [ 0.      26.34922]] dk = [-7.65797 -9.56295]
dk: [-0.29063 -0.36293]

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.18472 1.06491]
Valor da função: 4.65719
Gradiente: [3.85848 5.61886]
Norma do gradiente: 6.81611

Sistema:
[[11.72917  0.     ]
 [ 0.      11.72917]] dk = [-3.858

+ Levenberg-Marquardt

In [41]:
opt.LevenbergMarquardt(x0, rho=100)

**************************************************
                LEVENBERG-MARQUARDT
**************************************************

Aproximação inicial: [1 2]
Valor da função: 25.08554
Gradiente: [22.08554 24.08554]
Norma do gradiente: 32.67849

Sistema:
[[1167.88403 1067.88403]
 [1067.88403 1167.88403]] dk = [-22.08554 -24.08554]
dk: [-0.00033 -0.02033]

--------------------------------------------------
                     1ª iteração
--------------------------------------------------

x: [0.99967 1.97967]
Valor da função: 24.59346
Gradiente: [21.67435 23.63435]
Norma do gradiente: 32.06805

Sistema:
[[1128.36003 1028.36003]
 [1028.36003 1128.36003]] dk = [-21.67435 -23.63435]
dk: [-0.0007 -0.0203]

--------------------------------------------------
                     2ª iteração
--------------------------------------------------

x: [0.99897 1.95937]
Valor da função: 24.10305
Gradiente: [21.26392 23.18472]
Norma do gradiente: 31.45927

Sistema:
[[1089.68543  989.68543]
 [ 